In [3]:
import overpy
import geopandas as gpd
from shapely.geometry import Polygon, box
from tqdm import tqdm

In [ ]:
def get_land_use(bbox: box) -> gpd.GeoDataFrame:
    api = overpy.Overpass()
    query = f"""
    [out:json];
    wr["landuse"]({bbox.bounds[0]}, {bbox.bounds[1]}, {bbox.bounds[2]}, {bbox.bounds[3]});
    (._;>;);
    out body;
    """

    result = api.query(query)
    return result

    geoms = []
    land_use = []
    for way in tqdm(result.ways):
        try:
            coords = [(node.lon, node.lat) for node in way.nodes]
            
            if coords:
                polygon = Polygon(coords)
                geoms.append(polygon)
                land_use.append(way.tags.get("landuse"))
        except:
            pass
    
    return gpd.GeoDataFrame({"geometry": geoms, "landuse": land_use}, crs="EPSG:4326")

In [14]:
bbox = box(55.66731601212282, 12.560585920836248, 55.699960803118294, 12.607727632258538)
gdf = get_land_use(bbox)
# gdf.explore("landuse")
gdf

OverpassBadRequest: Error: line 3: parse error: ';' expected - 'wr' found. 